In [1]:
import os
import datetime
import pickle
import math

import scipy
import numpy  as np
import pandas as pd
import cv2

from matplotlib   import pyplot as plt

import tensorflow as     tf
from keras.models import Model
from keras.models import load_model
from keras.models import model_from_json
from keras.layers import Input
from PIL          import Image

from sklearn.metrics       import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.neighbors     import NearestNeighbors

import boto3
import zipfile

import attribute_utils
import dataset_utils
import prediction_utils

In [2]:
SHOW_MODEL = False

S3_BUCKET_NAME = "cs230project"
INPUTS_FNAME   = "lfw_datasets_and_models.zip"

PATH_INPUTS_FNAME     = "../../lfw_datasets_and_models.zip" 
PATH_INPUTS           = "../../lfw_datasets_and_models"

PATH_MODEL_FACENET_PRE = PATH_INPUTS + "/models/facenet/pretrained/model/facenet_keras.h5"

PATH_MODEL_FACENET_TRAINED = PATH_INPUTS + "/models/facenet/retrained/facenet_keras_trained.h5"

PATH_MODEL_FACENET_RET_ARCH    = PATH_INPUTS + "/models/facenet/retrained/encoding_facenet_arch.json"
PATH_MODEL_FACENET_RET_WEIGHTS = PATH_INPUTS + "/models/facenet/retrained/encoding_facenet_weights.h5"

PATH_MODEL_RESNET_ARCH    = PATH_INPUTS + "/models/resnet/encoding_resnet_arch.json"
PATH_MODEL_RESNET_WEIGHTS = PATH_INPUTS + "/models/resnet/encoding_resnet_weights.h5"

PATH_DATASET_BASE     = PATH_INPUTS + "/datasets/"

PATH_DATASET_BASE_MASKED   = PATH_DATASET_BASE + "/masked"
PATH_DATASET_BASE_UNMASKED = PATH_DATASET_BASE + "/unmasked"

PATH_DATASET_MASKED_TRAIN = PATH_DATASET_BASE_MASKED + "/train/"
PATH_DATASET_MASKED_VAL   = PATH_DATASET_BASE_MASKED + "/validation/"
PATH_DATASET_MASKED_TEST  = PATH_DATASET_BASE_MASKED + "/test/"

PATH_DATASET_UNMASKED_TRAIN = PATH_DATASET_BASE_UNMASKED + "/train/"
PATH_DATASET_UNMASKED_VAL   = PATH_DATASET_BASE_UNMASKED + "/validation/"
PATH_DATASET_UNMASKED_TEST  = PATH_DATASET_BASE_UNMASKED + "/test/"

PATH_DATASET_ATTRIBUTES = PATH_DATASET_BASE + "/attributes/lfw_attributes.csv"

PREDICT_THRESHOLD = 0.78125

MODEL = "facenet-pretrained"
# MODEL = "facenet-retrained"
# MODEL = "resnet"

# REQUIRED_SIZE = (224, 224)
REQUIRED_SIZE = (160, 160)


In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.5):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist   = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist   = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss       = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    return loss

In [4]:
# def batch_generator(batch_size=1):
#     y_val     = np.zeros((batch_size, 2, 1))
#     anchors   = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
#     positives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
#     negatives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    
#     anchors[0]   = image_a
#     positives[0] = image_p
#     negatives[0] = image_n
    
#     x_data = {
#         'anchor'         : anchors   ,
#         'anchorPositive' : positives ,
#         'anchorNegative' : negatives
#     }
    
#     yield (x_data, [y_val, y_val, y_val])
    
# #     while True:
# #         for i in range(batch_size):
# #             positiveFace = faces[np.random.randint(len(faces))]
# #             negativeFace = faces[np.random.randint(len(faces))]
# #             while positiveFace == negativeFace:
# #                 negativeFace = faces[np.random.randint(len(faces))]

# #             positives[i] = images[positiveFace][np.random.randint(len(images[positiveFace]))]
# #             anchors[i]   = images[positiveFace][np.random.randint(len(images[positiveFace]))]
# #             negatives[i] = images[negativeFace][np.random.randint(len(images[negativeFace]))]
        
# #         x_data = {'anchor': anchors,
# #                   'anchorPositive': positives,
# #                   'anchorNegative': negatives
# #                   }
        
        
        

In [5]:
def load_model_arch_weights(path_model_arch, path_model_weights):
    with open(path_model_arch, "r") as json_file:
        model_arch = json_file.read()

    #load the model architecture 
    model = model_from_json(model_arch)

    #load the model weights
    model.load_weights(path_model_weights)

    return model

In [6]:
def load_model_():
    ret = None
    if (MODEL == "facenet-pretrained"):
        print("[INFO] Loading Facenet pre-trained model...", PATH_MODEL_FACENET_PRE)
        ret = load_model(PATH_MODEL_FACENET_PRE)
        print('[INFO] Loaded Model')
    elif (MODEL == "facenet-retrained"):
        ret = load_model_arch_weights(PATH_MODEL_FACENET_RET_ARCH, PATH_MODEL_FACENET_RET_WEIGHTS)
    elif (MODEL == "resnet"):
        ret = load_model_arch_weights(PATH_MODEL_RESNET_ARCH, PATH_MODEL_RESNET_WEIGHTS)

    if (SHOW_MODEL):
        ret.summary()

    return ret

In [7]:
"""
Given a FaceNet model and an image
"""
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

In [8]:
facenet_model = load_model_()

[INFO] Loading Facenet pre-trained model... ../../lfw_datasets_and_models/models/facenet/pretrained/model/facenet_keras.h5
[INFO] Loaded Model


In [9]:
LAYERS_TO_FREEZE = 60
for layer in facenet_model.layers[0: LAYERS_TO_FREEZE]:
    layer.trainable  =  False

In [10]:
input_shape=(160, 160, 3)
A = Input(shape=input_shape, name = 'anchor')
P = Input(shape=input_shape, name = 'anchorPositive')
N = Input(shape=input_shape, name = 'anchorNegative')

In [11]:
enc_A = facenet_model(A)
enc_P = facenet_model(P)
enc_N = facenet_model(N)

In [12]:
tripletModel = Model([A, P, N], [enc_A, enc_P, enc_N])

In [13]:
tripletModel.compile(optimizer = 'adam', loss = triplet_loss)

In [14]:
# image_a = dataset_utils.extract_face(PATH_DATASET_MASKED_TRAIN + "/Aaron_Peirsol/Aaron_Peirsol_0001.jpg")
# image_p = dataset_utils.extract_face(PATH_DATASET_MASKED_TRAIN + "/Aaron_Peirsol/Aaron_Peirsol_0002.jpg")
# image_n = dataset_utils.extract_face(PATH_DATASET_MASKED_TRAIN + "/Zydrunas_Ilgauskas/Zydrunas_Ilgauskas_0001.jpg")

In [15]:
# gen = batch_generator(batch_size=16)
# tripletModel.fit_generator(gen, epochs=1, steps_per_epoch=1)

In [16]:
# emb_a = get_embedding(facenet_model, image_a)
# emb_p = get_embedding(facenet_model, image_p)
# emb_n = get_embedding(facenet_model, image_n)

In [17]:
# diff_a_p = tf.reduce_sum(tf.square(tf.subtract(emb_a, emb_p)), axis=-1)
# diff_a_n = tf.reduce_sum(tf.square(tf.subtract(emb_a, emb_n)), axis=-1)
# diff_p_n = tf.reduce_sum(tf.square(tf.subtract(emb_p, emb_n)), axis=-1)

In [18]:
# print(diff_a_p.numpy(), diff_a_n.numpy(), diff_p_n.numpy())

In [19]:
train_masked_x, train_masked_y = dataset_utils.load_dataset(PATH_DATASET_MASKED_TRAIN, REQUIRED_SIZE)

[DEBUG] Loading dataset... ../../lfw_datasets_and_models/datasets//masked/train/
i=0; subdir=Carl_Levin
i=500; subdir=Craig_Morgan
i=1000; subdir=Lawrence_Vito
i=1500; subdir=Mary_Katherine_Smart
i=2000; subdir=Bryan_Chui
i=2500; subdir=Luis_Gonzalez
i=3000; subdir=Ian_Wilmut
i=3500; subdir=Melvin_Talbert
i=4000; subdir=Ben_Kingsley
i=4500; subdir=Claudia_Pechstein
i=5000; subdir=Charmaine_Crooks
i=5500; subdir=Roberto_Laratro


In [20]:
val_masked_x, val_masked_y = dataset_utils.load_dataset(PATH_DATASET_MASKED_VAL, REQUIRED_SIZE)

[DEBUG] Loading dataset... ../../lfw_datasets_and_models/datasets//masked/validation/
i=0; subdir=Mohamed_ElBaradei
i=500; subdir=Torri_Edwards
i=1000; subdir=Tamara_Brooks
i=1500; subdir=Jean_Chretien


In [21]:
train_x = np.concatenate((train_masked_x, val_masked_x), axis=0)
train_y = np.concatenate((train_masked_y, val_masked_y), axis=0)

print(train_x.shape, train_y.shape)

(12223, 160, 160, 3) (12223,)


In [22]:
print("[INFO] Label encoding...")
print("Dataset: train=%d" % (train_x.shape[0]))

# label encode targets
out_encoder = LabelEncoder()
encoder_arr = np.append (train_y, ['Ben_Affleck', 'Ben_Curtis', 'Ben_Howland', 'Benazir_Bhutto', 'Benjamin_Netanyahu', 'Bernard_Landry', 'Bernard_Law', 'Bertie_Ahern'])
out_encoder.fit(encoder_arr)

train_enc_y = out_encoder.transform(train_y)
train_enc_y

[INFO] Label encoding...
Dataset: train=12223


array([ 757, 4485, 4261, ..., 2902, 5078, 1465])

In [23]:
"""
Given a FaceNet model and an image
"""
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

In [24]:
def get_embeddings(dataset, facenet_model):
    ret = list()
    i = 0
    for face in dataset:
        emd = get_embedding(facenet_model, face)
        ret.append(emd)
        if (i % 1000 == 0):
            print(i)
        i += 1
    ret = np.asarray(ret)
    return ret

In [25]:
# Build a dataframe of Person labels with the counts of images in the dataset for that person.
# df_train_enc_y        = pd.DataFrame(train_enc_y)
# df_train_enc_y        = df_train_enc_y.rename(columns = {0: "Person"})
# df_train_enc_counts_y = df_train_enc_y.groupby("Person").size().reset_index()
# df_train_enc_counts_y = df_train_enc_counts_y.rename(columns = {0: "Count"})

In [26]:
# df_train_enc_y

In [27]:
# df_train_enc_counts_y

In [28]:
def batch_generator(batch_size=16):
    y_val     = np.zeros((batch_size, 2, 1))
    anchors   = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    positives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    negatives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    
    while True:
        for i in range(batch_size):
            idx_anchor = np.random.randint(len(train_x[0:10]))
            idx_prev   = idx_anchor - 1
            idx_next   = idx_anchor + 1
            prs_anchor = train_enc_y[idx_anchor]

            if (idx_prev < 0):
                idx_prev = idx_anchor                

            if (idx_next >= len(train_x[0:10])):
                idx_next = idx_anchor

            if (train_enc_y[idx_prev] != prs_anchor):
                idx_prev = idx_anchor

            if (train_enc_y[idx_next] != prs_anchor):
                idx_next = idx_anchor

            idx_positive = idx_prev
            if (np.random.random() > 0.5):
                idx_positive = idx_next

            idx_negative = np.random.randint(len(train_x[0:10]))
            while ((idx_negative == idx_anchor) or (idx_negative == idx_positive)):
                idx_negative = np.random.randint(len(train_x[0:10]))

            if ((idx_anchor == 3) or (idx_anchor == 4)):
                print(idx_anchor, idx_positive, idx_negative, idx_prev, idx_next)
                
            anchors[i]   = train_x[idx_anchor]
            positives[i] = train_x[idx_positive]
            negatives[i] = train_x[idx_negative]
        
        x_data = {
            'anchor'         : anchors,
            'anchorPositive' : positives,
            'anchorNegative' : negatives
        }

        yield (x_data, [y_val, y_val, y_val])        


In [29]:
def batch_generator(batch_size=16):
    y_val     = np.zeros((batch_size, 2, 1))
    anchors   = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    positives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    negatives = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]))
    
    while True:
        for i in range(batch_size):
            anchors[i]   = train_x[3]
            positives[i] = train_x[4]
            negatives[i] = train_x[1]

        x_data = {
            'anchor'         : anchors,
            'anchorPositive' : positives,
            'anchorNegative' : negatives
        }

        yield (x_data, [y_val, y_val, y_val])        

In [30]:
gen = batch_generator(batch_size=16)
tripletModel.fit_generator(gen, epochs=1, steps_per_epoch=1)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1/1 [==============================] - 25s 25s/step - loss: 493.0129 - inception_resnet_v1_loss: 172.7406 - inception_resnet_v1_1_loss: 30.2065 - inception_resnet_v1_2_loss: 290.0658


In [31]:
facenet_model.save(PATH_MODEL_FACENET_TRAINED)

In [32]:
train_emd_x = get_embeddings(train_x[:50], facenet_model)

0


In [33]:
# normalize input vectors
in_encoder       = Normalizer(norm='l2')
train_emd_norm_x = in_encoder.transform(train_emd_x)
# val_emd_norm_x   = in_encoder.transform(val_emd_x)
# test_emd_norm_x  = in_encoder.transform(test_emd_x)

# train_emd_norm_x = train_emd_x

In [34]:
diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[3], train_emd_norm_x[4])), axis=-1)
print(diff.numpy())

diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[3], train_emd_norm_x[1])), axis=-1)
print(diff.numpy())

diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[1], train_emd_norm_x[4])), axis=-1)
print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[0], train_emd_norm_x[1])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[1], train_emd_norm_x[2])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[2], train_emd_norm_x[3])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[2], train_emd_norm_x[3])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[24], train_emd_norm_x[25])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[25], train_emd_norm_x[26])), axis=-1)
# print(diff.numpy())

# diff = tf.reduce_sum(tf.square(tf.subtract(train_emd_norm_x[27], train_emd_norm_x[28])), axis=-1)
# print(diff.numpy())



1.0575176
1.6898618
1.3890479
